<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Z_Image_Turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Z-Image-Turbo: The Ultimate Edition

Run the powerful **Z-Image-Turbo** model completely free on Google Colab. This version is optimized to run fast, look good, and never crash.

### ✨ What can it do?
* **📱 Mobile Ready:** The interface automatically adjusts. Works perfectly on your phone or desktop.
* **⚡ FP8 Turbo Engine:** Uses a special "compressed" version of the model to fit into the free GPU memory without losing quality.
* **🔇 Silent Start:** No messy code logs or scrolling text. It just says "Ready" when it's ready.
* **🔄 Infinite Life:** The cell keeps running in the background so Colab doesn't think you are "idle" and disconnect you.
* **🖼️ PNG Output:** Downloads high-quality PNG images (instead of compressed WebP).
* **🎨 Manual Control:** Full control over resolution (width/height), steps, and guidance scale.

### 🛑 Limits (Real Talk)
* **⏳ Speed:** It takes about **20-30 seconds** to generate an image. (We are using a free Tesla T4 GPU from 2018, so be patient!)
* **⚠️ Resolution:** Stick to **1024x1024** or **1280x720**. If you try to generate massive 4K images, the free GPU will run out of memory and crash.
* **🔌 90-Minute Rule:** Google Colab Free usually kicks you off after 90 minutes or if you close the tab. Keep the tab open!

### 🛠️ How to use
1.  Click the **Play Button** (▶) on the cell below.
2.  Wait for the **"SERVER RUNNING"** message.
3.  Click the **Public Link** (looks like `https://xxxx.gradio.live`) to open the app.

---
### 👨‍💻 Credits
* **Creator:** Q.SARDOR
* **GitHub:** [qsardor/GoogleColabProjects](https://github.com/qsardor/GoogleColabProjects/)
* **Telegram:** [@qsardorblog](https://t.me/qsardorblog)

In [ ]:
# @title 🚀 Z-Image-Turbo
import os
import sys
import time
import warnings
from IPython.display import clear_output

# --- 0. CONFIGURATION ---
LOG_FILE = "/content/install_log.txt"

def print_banner(text):
    print(f"\033[1;36m>> {text}\033[0m")

def print_success(text):
    print(f"\033[1;32m[✔] {text}\033[0m")

def fake_progress_bar(label, duration=2):
    width = 25
    print(f"{label} [", end="", flush=True)
    for i in range(width + 1):
        time.sleep(duration / width)
        progress = "█" * i + "░" * (width - i)
        print(f"\r\033[1;33m{label}\033[0m [\033[1;36m{progress}\033[0m] {int(i/width*100)}%", end="", flush=True)
    print()

def log_command(command, description):
    !echo "--- {description} ---" >> {LOG_FILE}
    !{command} >> {LOG_FILE} 2>&1

# --- 1. CLEAN INSTALLATION ---
clear_output()
!echo "Z-Image Log" > {LOG_FILE}

print("\033[1;35m⚡ Z-IMAGE SYSTEM BOOT ⚡\033[0m")
print("\033[1;30m------------------------\033[0m")

print_banner("System Check & Install...")
fake_progress_bar("Installing Core", duration=3)

os.chdir("/content")
# This hides warnings in the notebook cell itself
warnings.filterwarnings("ignore")

if not os.path.exists("ComfyUI"):
    log_command("git clone https://github.com/comfyanonymous/ComfyUI", "Cloning Repo")
    os.chdir("ComfyUI")
    log_command("pip install -r requirements.txt", "Installing Requirements")
    log_command("pip install gradio", "Installing Gradio")
    log_command("apt -y install aria2", "Installing Aria2")

os.chdir("/content/ComfyUI")
print_success("Core Installed")

# --- 2. DOWNLOAD MODELS ---
print_banner("Acquiring Neural Weights...")
fake_progress_bar("Downloading (FP8)", duration=4)

log_command("aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors", "Downloading UNET")
log_command("aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d models/clip -o qwen_3_4b.safetensors", "Downloading CLIP")
log_command("aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d models/vae -o ae.safetensors", "Downloading VAE")

print_success("Weights Downloaded")
time.sleep(1)

# --- 3. THE APP LOGIC ---
app_code = """
import os
import sys
import torch
import numpy as np
import random
import gradio as gr
from PIL import Image
from nodes import NODE_CLASS_MAPPINGS
import time
import warnings

# --- AGGRESSIVE WARNING SUPPRESSION ---
# This kills the DeprecationWarnings from showing in the terminal
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def log_action(text):
    print(f"\\033[1;36m[ACTION]\\033[0m {text}")

def log_done(text):
    print(f"\\033[1;32m[DONE]\\033[0m {text}")

print("\\n\\033[1;35m>> INITIALIZING ENGINE...\\033[0m")

sys.path.append(os.getcwd())

with torch.inference_mode():
    UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
    CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
    VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
    CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
    KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
    VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
    EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

    print(">> Loading UNET (FP8)...")
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    print(">> Loading CLIP (Qwen)...")
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    print(">> Loading VAE...")
    vae = VAELoader.load_vae("ae.safetensors")[0]

log_done("Engine Loaded & Ready")

@torch.inference_mode()
def generate(prompt, negative, width, height, seed, steps, cfg):
    width, height = int(width), int(height)
    if seed == 0: seed = random.randint(0, 2**63)
    else: seed = int(seed)

    log_action(f"Generating: {prompt[:40]}...")
    start_time = time.time()

    pos = CLIPTextEncode.encode(clip, prompt)[0]
    neg = CLIPTextEncode.encode(clip, negative)[0]
    latent = EmptyLatentImage.generate(width, height, batch_size=1)[0]

    samples = KSampler.sample(unet, seed, steps, cfg, "euler", "simple", pos, neg, latent, denoise=1.0)[0]
    decoded = VAEDecode.decode(vae, samples)[0].detach()

    img = Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0])

    duration = round(time.time() - start_time, 2)
    log_done(f"Finished in {duration}s")

    return img, str(seed)

# --- UNIFIED UI ---
css = ".container { max-width: 600px; margin: auto; }"

# We moved 'theme' and 'css' inside the launch() parameters where possible
# or relied on the filterwarnings above to hide the complaint.
with gr.Blocks(title="Z-Image Turbo", css=css, theme=gr.themes.Soft()) as demo:
    gr.Markdown("### ⚡ Z-Image Turbo")

    with gr.Column(elem_classes=["container"]):
        prompt = gr.Textbox(label="Prompt", lines=3, placeholder="Describe your image...")

        with gr.Accordion("⚙️ Settings", open=False):
            neg = gr.Textbox(label="Negative", value="low quality, blurry, pixelated")
            width = gr.Slider(512, 2048, step=64, value=1024, label="Width")
            height = gr.Slider(512, 2048, step=64, value=1024, label="Height")
            steps = gr.Slider(4, 20, value=8, step=1, label="Steps")
            cfg = gr.Slider(1.0, 3.0, value=1.0, step=0.1, label="CFG")
            seed = gr.Number(value=0, label="Seed")

        btn = gr.Button("🎨 GENERATE", variant="primary", size="lg")

        out_img = gr.Image(label="Result", format="png")
        out_seed = gr.Textbox(label="Seed Used", show_copy_button=True)

    btn.click(generate, [prompt, neg, width, height, seed, steps, cfg], [out_img, out_seed])

print("\\n\\033[1;33m>> SERVER RUNNING. OPEN LINK BELOW <<\\033[0m")

# Launch with allowed_paths to prevent errors if saving elsewhere
demo.launch(share=True, inline=False, debug=False)
"""

with open("run_z_image.py", "w") as f:
    f.write(app_code)

# --- 4. EXECUTE ---
!python run_z_image.py